# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
!pip install langchain_community langchain_experimental langchain_openai

In [2]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [3]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [6]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [7]:
pip install chromadb

In [32]:
from langchain.agents import AgentExecutor, Tool, AgentType, ZeroShotAgent, initialize_agent
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI
from langchain_community.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

# Define a prompt template for creating to-do lists for sustainable AI system design
todo_prompt = PromptTemplate.from_template(
    "You are a planner specializing in designing AI systems for a sustainable future. Come up with a to-do list for this objective: {objective}. Make sure the tasks are actionable and clear!"
)

# Create the to-do chain using OpenAI
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)

# Setup the search tool with SerpAPI for up-to-date information about sustainable AI systems
search = SerpAPIWrapper()

# Create tools for agent to use
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for answering questions about current events like sustainable AI systems, renewable energy, etc."
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="Useful for creating to-do lists for sustainable AI system design."
    ),
]

# Setup the prefix and suffix for the agent's reasoning
prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""

# Create a prompt for the agent that combines the objective, context, task, and agent scratchpad
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

# Example data for sustainable AI systems
sustainable_ai_data = [
    "Design AI systems that use renewable energy sources like solar and wind.",
    "Create AI models for optimizing waste management in cities.",
    "Develop AI solutions to monitor and reduce carbon emissions.",
    "Use AI for smart city planning to create more sustainable urban environments."
]

# Ensure tasks are created with the proper metadata ("task" key)
def create_task_with_metadata(task_description):
    return Document(
        page_content=task_description,
        metadata={"task": task_description}  # Add 'task' metadata explicitly
    )

# Convert the text data into a list of Documents, ensuring the 'task' field is set in metadata
documents = [create_task_with_metadata(text) for text in sustainable_ai_data]

# Setup the embeddings for the sustainable AI dataset
embeddings = OpenAIEmbeddings()

# Create the vector store for sustainable AI design documents
vectorstore = Chroma.from_documents(documents, embeddings)

# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

# Add the new tool for querying sustainable AI design data
tools.append(
    Tool(
        name="Sustainable AI Design QA",
        func=retriever.get_relevant_documents,
        description="Useful for answering questions related to AI systems for sustainability."
    )
)

# Initialize the language model (LLM)
llm = OpenAI(temperature=0)
# Initialize the agent with the tools and agent type (using initialize_agent)
# Initialize the agent with the tools and agent type (using initialize_agent)
agent = initialize_agent(
    tools=tools,
    llm=llm,  # Pass the language model explicitly
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Set agent type explicitly
    verbose=True
)

# Define the objective and other required inputs for the agent
objective = "Write a creative plan for developing a sustainable urban city using AI and renewable energy."
task = "What are the steps needed to design an AI system for sustainability?"
context = "Previous tasks include creating energy-efficient AI models and monitoring environmental impact with AI."

# Ensure that the agent receives all required input variables, including 'input'
input_query = "What are the steps to design an AI system for sustainability?"

# Execute the agent with the objective and necessary context
response = agent.invoke({
    "input": input_query,  # Added 'input' key here
    "objective": objective,
    "task": task,  # Added task input
    "context": context,  # Added context input
    "agent_scratchpad": ""  # Optional, you can pass this if necessary
})

# Print the response to see the result
print(response)




> Entering new AgentExecutor chain...
 Designing an AI system for sustainability requires careful planning and consideration of various factors.
Action: TODO
Action Input: Create a to-do list with tasks such as researching current sustainable AI systems, identifying potential ethical concerns, and consulting with experts in the field.
Observation: 

1. Research current sustainable AI systems and their impact on the environment and society.
2. Identify potential ethical concerns related to the development and implementation of AI systems.
3. Consult with experts in the field of sustainable AI to gain insights and gather diverse perspectives.
4. Conduct a thorough analysis of the current and future needs for sustainable AI systems.
5. Develop a framework for designing and evaluating sustainable AI systems.
6. Collaborate with stakeholders, including government agencies, NGOs, and businesses, to understand their needs and concerns.
7. Create a plan for integrating sustainable practices 

In [33]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [34]:
OBJECTIVE = "Write a creative plan for developing a sustainable urban city using AI and renewable energy."

In [35]:

# Logging of LLMChains
verbose = True  # Set to True to get more detailed output
max_iterations: Optional[int] = 3  # Limiting iterations to 3 for the objective

# Instantiate BabyAGI with the provided objective and necessary tools
baby_agi = BabyAGI.from_llm(
    llm=llm,  # OpenAI or another LLM model you are using
    vectorstore=vectorstore,  # The vector store where relevant data is stored
    task_execution_chain=agent_executor,  # Chain to execute tasks
    verbose=verbose,  # Set verbosity
    max_iterations=max_iterations,  # Set max iterations
)
